In [5]:
from twikit import Client
import re
import string
import os
import json

printable = set(string.printable)


def sanitize(text):
    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'@\w+ ', ' ', text)
    
    text = ''.join(filter(lambda x: x in printable, text))
    text = text.lower().strip().rstrip()
    return text

In [6]:
client = Client('en-US')

clientDict = {'JoeNuts836498': 'techno34',
             'dylan_berndt': 'techno34'}

In [7]:
seenIds = []
path = 'data/'
files = os.listdir(path)
for fileName in files:
    if not fileName.endswith('.json'):
        continue
    file = open(path + fileName)
    fileData = json.load(file)
    for key, tweet in fileData.items():
        if key not in seenIds:
            seenIds.append(key)

In [8]:
import json
from datetime import datetime
import time

while True:
    data = {}
    
    for userName in clientDict:
        client.login(auth_info_1=userName, password=clientDict[userName])
        client.save_cookies('cookies.json')
        client.load_cookies(path='cookies.json')
    
        tweets = client.get_timeline(count=400, seen_tweet_ids=seenIds)
    
        usefulCount = 0
        replyCount = 0
        
        try:
            for tweet in tweets:
                if not tweet:
                    continue
                if tweet.id in seenIds:
                    continue
                seenIds.append(tweet.id)
                t = client.get_tweet_by_id(tweet.id)
                if not t.replies:
                    continue
                if len(t.replies) > 0:
                    usefulCount += 1
                    replyCount += len(t.replies)
                    sanitized = [sanitize(reply.text) for reply in t.replies]
                    replies = [reply for reply in sanitized if len(reply) > 0]
                    tweetData = {"text": sanitize(t.text), "replies": replies}
                    data[t.id] = tweetData
                    print(userName, "Tweets:", usefulCount, "Replies:", replyCount, end='\r')
        except Exception as e:
            print(e)
            continue
        print()
    
    if len(data) < 1:
        quit()
    
    now = datetime.now()
    timeName = now.strftime("%y.%m.%d.%H.%M.%S")
    
    file = open("data/timeline " + timeName + ".json", 'w+')
    json.dump(data, file, indent=4)
    file.close()

    time.sleep(600)

JoeNuts836498 Tweets: 30 Replies: 267
dylan_berndt Tweets: 28 Replies: 250
JoeNuts836498 Tweets: 30 Replies: 254
dylan_berndt Tweets: 32 Replies: 288
JoeNuts836498 Tweets: 30 Replies: 279
dylan_berndt Tweets: 32 Replies: 301
JoeNuts836498 Tweets: 31 Replies: 286
dylan_berndt Tweets: 33 Replies: 259
JoeNuts836498 Tweets: 31 Replies: 285
dylan_berndt Tweets: 27 Replies: 251
JoeNuts836498 Tweets: 30 Replies: 277
dylan_berndt Tweets: 29 Replies: 270
JoeNuts836498 Tweets: 31 Replies: 294
dylan_berndt Tweets: 32 Replies: 290
JoeNuts836498 Tweets: 31 Replies: 280
'NoneType' object has no attribute 'replies'
JoeNuts836498 Tweets: 30 Replies: 277
dylan_berndt Tweets: 27 Replies: 254
JoeNuts836498 Tweets: 31 Replies: 271
dylan_berndt Tweets: 28 Replies: 262


KeyboardInterrupt: 